In [ ]:
import math

class Value:
    def __init__(self, data, _children=(), _op='', label=''):
        self.data = data
        self.grad = 0
        self._prev = set(_children)
        self._op = _op
        self.label = label

    def __repr__(self):
        return f"Value(data={self.data})"

    def __add__(self, other):
        res = Value(self.data + other.data, (self, other), '+')
        return res

    def __mul__(self, other):
        res = Value(self.data * other.data, (self, other), '*')
        return res

    def tanh(self):
        n = self.data
        t = (math.exp(2 * n) - 1) / (math.exp(2 * n) + 1)
        res = Value(t, (self, ), 'tanh')
        return res

    def back(self):
        self.grad = 1.0
        visited = set()
        def propogate(v):
            visited.add(v)
            if v._op == '+':
                for child in v._prev:
                    child.grad += v.grad
            elif v._op == '*':
                a, b = tuple(v._prev)
                a.grad += v.grad * b.data
                b.grad += v.grad * a.data
            elif v._op == 'tanh':
                a, = tuple(v._prev)
                n = a.data
                t = (math.exp(2 * n) - 1) / (math.exp(2 * n) + 1)
                a.grad += (1 - t * t) * v.grad;
                
            for child in v._prev:
                if child not in visited:
                    propogate(child)
        propogate(self)

In [ ]:
from graphviz import Digraph

def trace(root):
    nodes, edges = set(), set()
    def build(v):
        if v not in nodes:
            nodes.add(v)
            for child in v._prev:
                edges.add((child, v))
                build(child)
    build(root)
    return nodes, edges

def draw_dot(root):
    dot = Digraph(format='svg', graph_attr={'rankdir': 'LR'})

    nodes, edges = trace(root)
    for n in nodes:
        uid = str(id(n))
        dot.node(name = uid, label = "{ %s | data %.4f | grad %.4f}" % (n.label, n.data, n.grad), shape='record')
        if n._op:
            dot.node(name = uid + n._op, label = n._op)
            dot.edge(uid + n._op, uid)

    for n1, n2 in edges:
        uid1 = str(id(n1))
        uid2 = str(id(n2))
        dot.edge(uid1, uid2 + n2._op)

    return dot

In [ ]:
a = Value(2.0, label='a')
b = Value(-3.0, label='b')
c = Value(10.0, label='c')

e = a * b; e.label = 'e'
d = e + c; d.label = 'd'
f = Value(-2.0, label='f')
L = d * f; L.label = 'L'

L.back()

draw_dot(L)

In [ ]:
def bar():
    a5 = Value(2.0, label='a')
    b5 = Value(3.0, label='b')
    c5 = Value(-1.0, label='c')
    
    d5 = a + b; d.label = 'd'
    e5 = a + c; e.label = 'e'
    L5 = d + e; L.label = 'L'
    
    '''
    L = (a + b) + (a + c)
    dL/da = dL/da(2a + b + c) = 2
    
    L = ab + ac
    dL/da = dL/da(a(b + c)) = b + c
    
    L = (a + b) * (a + c)
    dL/da = dL/da(a^2 + ab + ac + bc) = 2a + b + c
    our method: dL/da = (a + c), dL/da = (b + c), add both gets the truth
    
    so in a case where a node has 2 parents, the derivative would be the sum
    '''

    L5.back()
    draw_dot(L5)

a5 = Value(2.0, label='a')
b5 = Value(3.0, label='b')
c5 = Value(-1.0, label='c')

d5 = a5 + b5; d5.label = 'd'
e5 = a5 + c5; e5.label = 'e'
L5 = d5 + e5; L5.label = 'L'

L5.back()
draw_dot(L5)

In [ ]:
def foo():
    h = 0.0001
    
    a = Value(2.0, label='a')
    b = Value(-3.0, label='b')
    c = Value(10.0, label='c')
    
    e = a * b; e.label = 'e'
    d = e + c; d.label = 'd'
    f = Value(-2.0, label='f')
    L1 = d * f; L.label = 'L'

    a = Value(2.0, label='a')
    b = Value(-3.0, label='b')
    c = Value(10.0 + h, label='c')
    
    e = a * b; e.label = 'e'
    d = e + c; d.label = 'd'
    f = Value(-2.0, label='f')
    L2 = d * f; L.label = 'L'

    print((L2.data - L1.data) / h)

foo()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

plt.plot(np.arange(-5, 5, 0.2), np.tanh(np.arange(-5, 5, 0.2)))
plt.grid()

In [ ]:
# inputs
x1 = Value(2.0, label='x1')
x2 = Value(0.0, label='x2')

# weights
w1 = Value(-3.0, label='w1')
w2 = Value(1.0, label='w2')

# bias
b = Value(6.8813735870195432, label='b')

# neuron = x1 * w1 + x2 * w2 + b
x1w1 = x1 * w1; x1w1.label = 'x1w1'
x2w2 = x2 * w2; x2w2.label = 'x2w2'
x1w1x2w2 = x1w1 + x2w2; x1w1x2w2.label = 'x1w1x2w2'
n = x1w1x2w2 + b; n.label = 'n'

o = n.tanh(); o.label='o'
o.back()

draw_dot(o)